In [3]:
from photerr import LsstErrorModel
import pandas as pd
import numpy as np

In [5]:
noiseless_photometry = np.load("/Users/fpetri/repos/LBGforecast/data/data/training_data.npy")[:1000000, :6]
sps_params = np.load("/Users/fpetri/repos/LBGforecast/data/data/training_params.npy")[:1000000, :]
catalog = pd.DataFrame(noiseless_photometry, columns=['u', 'g', 'r', 'i', 'z', 'y'])

# remove photometry brighter than brightness cut
brightness_cut = 18
for column in ['u','g','r','i','z','y']:
    catalog = catalog.drop(catalog[catalog[column] < brightness_cut].index)

catalog

,u,g,r,i,z,y
0,35.979999,34.826707,33.827043,30.591961,29.763766,24.299834
1,20.662052,20.634097,20.163877,20.307578,20.376406,20.447159
2,27.477763,26.960422,26.893122,26.893755,26.835643,26.860594
3,36.451890,35.415322,34.327853,32.541182,31.546398,30.657177
4,18.311344,18.285075,18.000323,18.051494,18.085408,18.033818
...,...,...,...,...,...,...
999994,22.466823,21.958965,21.512976,20.850251,20.974092,20.654374
999995,40.123789,38.240046,27.177094,24.436528,23.433175,22.784754
999997,28.242125,23.668817,22.560040,21.593582,20.983335,20.537410
999998,43.036269,37.746546,29.504046,27.274358,26.449181,26.099206


In [11]:
errModel = LsstErrorModel(sigLim=0)
sig5detections = LsstErrorModel(sigLim=5)
sig2detections = LsstErrorModel(sigLim=2)

observed_catalog = errModel(catalog, random_state=42).filter(['u', 'g', 'r', 'i', 'z', 'y']).replace([np.inf, -np.inf], np.nan, inplace=False)
observed_catalog.dropna(axis=0, inplace=True) #remove non detections due to negative fluxes

catalog_sig5 = sig5detections(observed_catalog, random_state=42).filter(['u', 'g', 'r', 'i', 'z', 'y']).replace([np.inf, -np.inf], np.nan, inplace=False)
catalog_sig5.rename(columns={"u": "u5", "g": "g5", "r": "r5", "i": "i5", "z": "z5", "y": "y5" }, inplace=True)

catalog_sig2 = sig2detections(observed_catalog, random_state=42).filter(['u', 'g', 'r', 'i', 'z', 'y']).replace([np.inf, -np.inf], np.nan, inplace=False)
catalog_sig2.rename(columns={"u": "u2", "g": "g2", "r": "r2", "i": "i2", "z": "z2", "y": "y2" }, inplace=True)

In [13]:
observed_catalog

,u,g,r,i,z,y
1,20.661355,20.635699,20.163962,20.311902,20.371887,20.442743
2,27.425175,26.793837,26.826197,27.103724,26.704071,28.643803
4,18.313505,18.286843,17.997658,18.049663,18.083339,18.031653
11,22.894148,22.003180,20.552170,21.234631,20.002153,21.202984
16,25.651666,24.259360,23.585921,23.250018,23.067536,22.958524
...,...,...,...,...,...,...
999991,28.239119,28.348679,27.668265,25.892125,25.565473,24.517055
999993,28.796676,27.730703,26.663461,26.006994,24.442110,24.057591
999994,22.466220,21.953191,21.509219,20.848127,20.973475,20.644407
999997,26.879317,23.672854,22.564309,21.596926,20.978265,20.522467
